In [1]:
#!/usr/bin/python 
# -*- coding: utf-8 -*-

import torch
import torchvision
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.utils.data import Dataset
import os
from PIL import Image
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import StepLR
from PIL import Image
import torchxrayvision as xrv
import torch.nn.functional as F
import torch.nn as nn
import numpy as np
from sklearn.metrics import roc_auc_score

torch.cuda.empty_cache()

In [2]:
transform = transforms.Compose([xrv.datasets.XRayCenterCrop(),
                                        xrv.datasets.XRayResizer(224)])

In [3]:
bs = 4

In [4]:
covid19 = xrv.datasets.COVID19_Dataset(
            imgpath='covid_data/images',
            csvpath='covid_data/metadata.csv',
            transform=transform)

# count split sizes
n_train = int(0.8 * len(covid19))
# n_valid_test = len(covid19) - n_train
# n_valid = int(0.5 * n_valid_test)
# n_test = n_valid_test - n_valid
n_test = len(covid19) - n_train

# print(f'Covid Chest x-ray stats dataset stats:\n\n{covid19},\nnumber of training={n_train}, \
# \n number of validing={n_valid} ,\n number of testing={n_test}',flush=True)
print(f'Covid Chest x-ray stats dataset stats:\n\n{covid19},\nnumber of training={n_train}, \
,\n number of testing={n_test}',flush=True)
# split the dataset
# train_set, val_set, test_set = torch.utils.data.random_split(covid19, [n_train, n_valid, n_test])
train_set, test_set = torch.utils.data.random_split(covid19, [n_train, n_test])

{'ARDS': {0.0: 88, 1.0: 4},
 'Bacterial Pneumonia': {0.0: 86, 1.0: 6},
 'COVID-19': {0.0: 23, 1.0: 69},
 'MERS': {0.0: 92},
 'No Finding': {0.0: 91, 1.0: 1},
 'Pneumonia': {0.0: 2, 1.0: 90},
 'SARS': {0.0: 81, 1.0: 11},
 'Streptococcus': {0.0: 86, 1.0: 6},
 'Viral Pneumonia': {0.0: 12, 1.0: 80}}
Covid Chest x-ray stats dataset stats:

COVID19_Dataset num_samples=92,
number of training=73, ,
 number of testing=19


In [5]:
labeltest=covid19[1]['lab']
print(labeltest[[8]])

[1.]


In [6]:
def load_train_data():
    train_loader = torch.utils.data.DataLoader(train_set, batch_size = bs, num_workers = 0, shuffle=True)
    return train_loader

def load_val_data():
    test_loader = torch.utils.data.DataLoader(val_set, batch_size = bs, num_workers = 0, shuffle=False)
    return test_loader

def load_test_data():
    test_loader = torch.utils.data.DataLoader(test_set, batch_size = bs, num_workers = 0, shuffle=False)
    return test_loader

In [7]:

# train_data = xrv.datasets.COVID19_Dataset('covid_data/images', transform=transform)
# # print(train_data[1]["PA"])
# train_loader = torch.utils.data.DataLoader(train_data, batch_size = 64, num_workers = 0, shuffle=True)
# for idx, batch_samples in enumerate(train_loader):
#     print(1)
#     print(idx)
#     text_batchs, text_labels = batch_samples["PA"], batch_samples["lab"]
#     print(text_batchs)
# # dataiter=iter(train_loader)
# # print(dataiter)
# # data=dataiter.next
# # print(data["PA"])

# # images=data
# # print(data)
# # print(type(images),type(labels))
# # print(type(train_loader))

In [8]:
class DenseNetModel(nn.Module):

    def __init__(self):
        """
        Pass in parsed HyperOptArgumentParser to the model
        :param hparams:
        """
        super(DenseNetModel, self).__init__()

        self.dense_net = xrv.models.DenseNet(num_classes=2)
        self.criterion = nn.CrossEntropyLoss()

    def forward(self, x):
        logits = self.dense_net(x)
        return logits

In [9]:
class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()           
            
        self.conv1 = nn.Conv2d(1, 64, kernel_size=11, stride=4, padding=2)
        self.conv2 = nn.Conv2d(64, 192, kernel_size=5, padding=2)
        self.conv3 = nn.Conv2d(192, 384, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(384, 256, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        
        self.dropout = nn.Dropout(0.5)        
        self.activation = nn.RReLU(inplace = True)
        
        self.pool = nn.AdaptiveAvgPool2d((6,6))
        
        self.dense1 = nn.Linear(256*6*6, 4096)
        self.dense2 = nn.Linear(4096, 4096)
        self.out = nn.Linear(4096, 2)
        
    def forward(self, x):

        x = self.conv1(x)
        x = self.activation(x)
        x = F.max_pool2d(x, kernel_size=3, stride = 2)
        
        x = self.conv2(x)
        x = self.activation(x)
        x = F.max_pool2d(x, kernel_size=3, stride = 2)
        
        x = self.conv3(x)
        x = self.activation(x)
        
        x = self.conv4(x)
        x = self.activation(x)
        
        x = self.conv5(x)
        x = self.activation(x)
        x = F.max_pool2d(x, kernel_size=3, stride = 2)
        
        ########
        
        x = self.pool(x)
        x = torch.flatten(x, 1)
        
        x = self.dropout(x)
        x = self.dense1(x)
        x = self.activation(x)
        

        x = self.dropout(x) 
        x = self.dense2(x)
        x = self.activation(x)
        x = F.log_softmax(x, dim=1)
        
        return x

In [10]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

def train(optimizer, epoch):
    
    model.train()
    train_loader = load_train_data()
    
    train_loss = 0
    train_correct = 0
    
    for batch_index, batch_samples in enumerate(train_loader):
        
        # move data to device
        data, target = batch_samples["PA"].to(device), batch_samples["lab"].to(device)
        
        optimizer.zero_grad()
        output = model(data)
        
#         print(output)
#         print(target.long()[:, 2])
#         loss = nn.functional.nll_loss(output, target.long()[:, 2])
        criteria = nn.CrossEntropyLoss()
        loss = criteria(output, target.long()[:, 2])
        train_loss += criteria(output, target.long()[:, 2])
        
        loss.backward()
        optimizer.step()
        
        pred = output.argmax(dim=1, keepdim=True)
        train_correct += pred.eq(target.long()[:, 2].view_as(pred)).sum().item()
    
        # Display progress and write to tensorboard
        if batch_index % bs == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tTrain Loss: {:.6f}'.format(
                epoch, batch_index, len(train_loader),
                100.0 * batch_index / len(train_loader), loss.item()/ bs))
            
#             niter = epoch*len(train_loader)+batch_index
#             writer.add_scalar('Train/Loss', loss.data, niter)
    
    print('\nTrain set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        train_loss/len(train_loader.dataset), train_correct, len(train_loader.dataset),
        100.0 * train_correct / len(train_loader.dataset)))

In [14]:
def test(epoch):
    
    model.eval()
    test_loss = 0
    correct = 0
    test_loader = load_test_data()
    results = []
    
    TP = 0
    TN = 0
    FN = 0
    FP = 0
    
    
    criteria = nn.CrossEntropyLoss()
    # Don't update model
    with torch.no_grad():
        tpr_list = []
        fpr_list = []
        
        predlist=[]
        targetlist=[]
        # Predict
        for batch_index, batch_samples in enumerate(test_loader):
            
            data, target = batch_samples["PA"].to(device), batch_samples["lab"].to(device)
            output = model(data)
            
            
            test_loss += criteria(output, target.long()[:, 2])
#             test_loss += F.nll_loss(output, target.long()[:, 2], reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
#             print(pred)
#             print(target.long()[:, 2].view_as(pred))
            correct += pred.eq(target.long()[:, 2].view_as(pred)).sum().item()
#             print(pred.eq(target.long()[:, 2].view_as(pred)))
            TP += ((pred == 1) & (target.long()[:, 2].view_as(pred).data == 1)).cpu().sum()
            TN += ((pred == 0) & (target.long()[:, 2].view_as(pred) == 0)).cpu().sum()
#             # FN    predict 0 label 1
            FN += ((pred == 0) & (target.long()[:, 2].view_as(pred) == 1)).cpu().sum()
#             # FP    predict 1 label 0
            FP += ((pred == 1) & (target.long()[:, 2].view_as(pred) == 0)).cpu().sum()
            print(TP,TN,FN,FP)
            
            
#             print(output[:,1].cpu().numpy())
#             print((output[:,1]+output[:,0]).cpu().numpy())
            predcpu=(output[:,1].cpu().numpy())/((output[:,1]+output[:,0]).cpu().numpy())
            targetcpu=target.long()[:, 2].cpu().numpy()
            predlist=np.append(predlist, predcpu)
            targetlist=np.append(targetlist,targetcpu)
#             print('pred',predlist)
# #             print('pred2',pred)
#             print('target',targetlist)
            
        print('TP=',TP,'TN=',TN,'FN=',FN,'FP=',FP)
        print('TP+FP',TP+FP)
        p = TP.item() / (TP + FP).item()
        print('precision',p)
        p = TP.item() / (TP + FP).item()
        r = TP.item() / (TP + FN).item()
        print('recall',r)
        F1 = 2 * r * p / (r + p)
        acc = (TP + TN).item() / (TP + TN + FP + FN).item()
        print('F1',F1)
        print('acc',acc)
#         print('pred',predlist)
#         print('target',targetlist)
#         print('pred',predcpu)
#         print('target',targetcpu)
#         print('AUC',roc_auc_score(targetcpu,predcpu))
        print('AUC',roc_auc_score(targetlist,predlist))
#         try:
#             print('AUC',roc_auc_score(predlist, targetlist))
#         except ValueError:
#             pass
#         tpr = tp/(tp+fn)
#         fpr = fp/(fp+tn)
#         tpr_list.append(tpr)
#         fpr_list.append(fpr)
#         fpr=TP.item() / (FP + TN).item()
#         tpr_list.append(r)
#         fpr_list.append(fpr)
#         print('AUC',np.trapz(tpr_list, fpr_list))
    test_loss /= len(test_loader.dataset)
    

    # Display results
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100.0 * correct / len(test_loader.dataset)))
    
#     return (100.0 * correct / len(test_loader.dataset))
    
    # Write to tensorboard
#     writer.add_scalar('Test Accuracy', 100.0 * correct / len(test_loader.dataset), epoch)

In [15]:
device = 'cuda'

# Tensorboard Writer
# model = CNN().to(device)
model = DenseNetModel().to(device)
    
#optimizer = optim.SGD(model.parameters(), lr=0.001, momentum = 0.9)
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = StepLR(optimizer, step_size=1)

for epoch in range(1, 100+1):
    train(optimizer, epoch)
    
    test(epoch)

    # Save model
#     torch.save(model.state_dict(), "model_backup/AlexNet_3_class_xray_cnn_{}.pt".format(epoch))  


# state = {'epoch': epoch + 1, 'state_dict': model.state_dict(),
#                  'optimizer': optimizer.state_dict(), 'scheduler' : scheduler}
# torch.save(state, "model_backup/AlexNetAdamState")

Train Epoch: 1 [0/19 (0%)]	Train Loss: 0.150896
Train Epoch: 1 [4/19 (21%)]	Train Loss: 0.185697
Train Epoch: 1 [8/19 (42%)]	Train Loss: 0.076104
Train Epoch: 1 [12/19 (63%)]	Train Loss: 0.236814
Train Epoch: 1 [16/19 (84%)]	Train Loss: 0.088191

Train set: Average loss: 0.1488, Accuracy: 55/73 (75%)

tensor(3) tensor(0) tensor(0) tensor(1)
tensor(5) tensor(1) tensor(0) tensor(2)
tensor(9) tensor(1) tensor(0) tensor(2)
tensor(11) tensor(1) tensor(0) tensor(4)
tensor(13) tensor(1) tensor(0) tensor(5)
TP= tensor(13) TN= tensor(1) FN= tensor(0) FP= tensor(5)
TP+FP tensor(18)
precision 0.7222222222222222
recall 1.0
F1 0.8387096774193548
acc 0.7368421052631579
AUC 0.5

Test set: Average loss: 0.1416, Accuracy: 14/19 (74%)

Train Epoch: 2 [0/19 (0%)]	Train Loss: 0.063784
Train Epoch: 2 [4/19 (21%)]	Train Loss: 0.083055
Train Epoch: 2 [8/19 (42%)]	Train Loss: 0.082180
Train Epoch: 2 [12/19 (63%)]	Train Loss: 0.060942
Train Epoch: 2 [16/19 (84%)]	Train Loss: 0.076581

Train set: Average loss: 

Train Epoch: 12 [4/19 (21%)]	Train Loss: 0.058886
Train Epoch: 12 [8/19 (42%)]	Train Loss: 0.066597
Train Epoch: 12 [12/19 (63%)]	Train Loss: 0.083127
Train Epoch: 12 [16/19 (84%)]	Train Loss: 0.054436

Train set: Average loss: 0.0845, Accuracy: 65/73 (89%)

tensor(3) tensor(1) tensor(0) tensor(0)
tensor(4) tensor(3) tensor(1) tensor(0)
tensor(8) tensor(3) tensor(1) tensor(0)
tensor(10) tensor(5) tensor(1) tensor(0)
tensor(12) tensor(6) tensor(1) tensor(0)
TP= tensor(12) TN= tensor(6) FN= tensor(1) FP= tensor(0)
TP+FP tensor(12)
precision 1.0
recall 0.9230769230769231
F1 0.9600000000000001
acc 0.9473684210526315
AUC 0.5

Test set: Average loss: 0.0652, Accuracy: 18/19 (95%)

Train Epoch: 13 [0/19 (0%)]	Train Loss: 0.013141
Train Epoch: 13 [4/19 (21%)]	Train Loss: 0.040769
Train Epoch: 13 [8/19 (42%)]	Train Loss: 0.020952
Train Epoch: 13 [12/19 (63%)]	Train Loss: 0.061563
Train Epoch: 13 [16/19 (84%)]	Train Loss: 0.031045

Train set: Average loss: 0.0647, Accuracy: 65/73 (89%)

tensor(3

Train Epoch: 23 [0/19 (0%)]	Train Loss: 0.012194
Train Epoch: 23 [4/19 (21%)]	Train Loss: 0.029856
Train Epoch: 23 [8/19 (42%)]	Train Loss: 0.045509
Train Epoch: 23 [12/19 (63%)]	Train Loss: 0.031632
Train Epoch: 23 [16/19 (84%)]	Train Loss: 0.027967

Train set: Average loss: 0.0214, Accuracy: 73/73 (100%)

tensor(3) tensor(1) tensor(0) tensor(0)
tensor(4) tensor(3) tensor(1) tensor(0)
tensor(8) tensor(3) tensor(1) tensor(0)
tensor(10) tensor(5) tensor(1) tensor(0)
tensor(12) tensor(6) tensor(1) tensor(0)
TP= tensor(12) TN= tensor(6) FN= tensor(1) FP= tensor(0)
TP+FP tensor(12)
precision 1.0
recall 0.9230769230769231
F1 0.9600000000000001
acc 0.9473684210526315
AUC 0.46153846153846156

Test set: Average loss: 0.0386, Accuracy: 18/19 (95%)

Train Epoch: 24 [0/19 (0%)]	Train Loss: 0.041722
Train Epoch: 24 [4/19 (21%)]	Train Loss: 0.002985
Train Epoch: 24 [8/19 (42%)]	Train Loss: 0.009260
Train Epoch: 24 [12/19 (63%)]	Train Loss: 0.049795
Train Epoch: 24 [16/19 (84%)]	Train Loss: 0.049645

Train Epoch: 34 [4/19 (21%)]	Train Loss: 0.003278
Train Epoch: 34 [8/19 (42%)]	Train Loss: 0.017126
Train Epoch: 34 [12/19 (63%)]	Train Loss: 0.002733
Train Epoch: 34 [16/19 (84%)]	Train Loss: 0.026939

Train set: Average loss: 0.0166, Accuracy: 73/73 (100%)

tensor(3) tensor(1) tensor(0) tensor(0)
tensor(4) tensor(3) tensor(1) tensor(0)
tensor(8) tensor(3) tensor(1) tensor(0)
tensor(10) tensor(5) tensor(1) tensor(0)
tensor(12) tensor(6) tensor(1) tensor(0)
TP= tensor(12) TN= tensor(6) FN= tensor(1) FP= tensor(0)
TP+FP tensor(12)
precision 1.0
recall 0.9230769230769231
F1 0.9600000000000001
acc 0.9473684210526315
AUC 0.6538461538461539

Test set: Average loss: 0.0395, Accuracy: 18/19 (95%)

Train Epoch: 35 [0/19 (0%)]	Train Loss: 0.018746
Train Epoch: 35 [4/19 (21%)]	Train Loss: 0.001863
Train Epoch: 35 [8/19 (42%)]	Train Loss: 0.024366
Train Epoch: 35 [12/19 (63%)]	Train Loss: 0.024036
Train Epoch: 35 [16/19 (84%)]	Train Loss: 0.004547

Train set: Average loss: 0.0130, Accuracy: 73/73

Train Epoch: 45 [4/19 (21%)]	Train Loss: 0.013753
Train Epoch: 45 [8/19 (42%)]	Train Loss: 0.001156
Train Epoch: 45 [12/19 (63%)]	Train Loss: 0.103938
Train Epoch: 45 [16/19 (84%)]	Train Loss: 0.016001

Train set: Average loss: 0.0176, Accuracy: 72/73 (99%)

tensor(3) tensor(0) tensor(0) tensor(1)
tensor(4) tensor(2) tensor(1) tensor(1)
tensor(8) tensor(2) tensor(1) tensor(1)
tensor(10) tensor(4) tensor(1) tensor(1)
tensor(12) tensor(5) tensor(1) tensor(1)
TP= tensor(12) TN= tensor(5) FN= tensor(1) FP= tensor(1)
TP+FP tensor(13)
precision 0.9230769230769231
recall 0.9230769230769231
F1 0.9230769230769231
acc 0.8947368421052632
AUC 0.5

Test set: Average loss: 0.0548, Accuracy: 17/19 (89%)

Train Epoch: 46 [0/19 (0%)]	Train Loss: 0.029481
Train Epoch: 46 [4/19 (21%)]	Train Loss: 0.005276
Train Epoch: 46 [8/19 (42%)]	Train Loss: 0.002038
Train Epoch: 46 [12/19 (63%)]	Train Loss: 0.022168
Train Epoch: 46 [16/19 (84%)]	Train Loss: 0.002815

Train set: Average loss: 0.0132, Accuracy: 73/73 

Train Epoch: 56 [8/19 (42%)]	Train Loss: 0.009045
Train Epoch: 56 [12/19 (63%)]	Train Loss: 0.015562
Train Epoch: 56 [16/19 (84%)]	Train Loss: 0.017068

Train set: Average loss: 0.0621, Accuracy: 72/73 (99%)

tensor(3) tensor(1) tensor(0) tensor(0)
tensor(5) tensor(3) tensor(0) tensor(0)
tensor(9) tensor(3) tensor(0) tensor(0)
tensor(11) tensor(4) tensor(0) tensor(1)
tensor(13) tensor(5) tensor(0) tensor(1)
TP= tensor(13) TN= tensor(5) FN= tensor(0) FP= tensor(1)
TP+FP tensor(14)
precision 0.9285714285714286
recall 1.0
F1 0.962962962962963
acc 0.9473684210526315
AUC 0.6923076923076923

Test set: Average loss: 0.0292, Accuracy: 18/19 (95%)

Train Epoch: 57 [0/19 (0%)]	Train Loss: 0.001087
Train Epoch: 57 [4/19 (21%)]	Train Loss: 0.017116
Train Epoch: 57 [8/19 (42%)]	Train Loss: 0.052728
Train Epoch: 57 [12/19 (63%)]	Train Loss: 0.040260
Train Epoch: 57 [16/19 (84%)]	Train Loss: 0.005208

Train set: Average loss: 0.0509, Accuracy: 72/73 (99%)

tensor(3) tensor(0) tensor(0) tensor(1)
tens

Train Epoch: 67 [16/19 (84%)]	Train Loss: 0.013041

Train set: Average loss: 0.0477, Accuracy: 72/73 (99%)

tensor(3) tensor(0) tensor(0) tensor(1)
tensor(5) tensor(2) tensor(0) tensor(1)
tensor(9) tensor(2) tensor(0) tensor(1)
tensor(11) tensor(3) tensor(0) tensor(2)
tensor(13) tensor(4) tensor(0) tensor(2)
TP= tensor(13) TN= tensor(4) FN= tensor(0) FP= tensor(2)
TP+FP tensor(15)
precision 0.8666666666666667
recall 1.0
F1 0.9285714285714286
acc 0.8947368421052632
AUC 0.42307692307692313

Test set: Average loss: 0.0298, Accuracy: 17/19 (89%)

Train Epoch: 68 [0/19 (0%)]	Train Loss: 0.001291
Train Epoch: 68 [4/19 (21%)]	Train Loss: 0.000664
Train Epoch: 68 [8/19 (42%)]	Train Loss: 0.001995
Train Epoch: 68 [12/19 (63%)]	Train Loss: 0.002970
Train Epoch: 68 [16/19 (84%)]	Train Loss: 0.030688

Train set: Average loss: 0.0439, Accuracy: 72/73 (99%)

tensor(3) tensor(0) tensor(0) tensor(1)
tensor(5) tensor(2) tensor(0) tensor(1)
tensor(9) tensor(2) tensor(0) tensor(1)
tensor(11) tensor(4) te

Train Epoch: 78 [16/19 (84%)]	Train Loss: 0.001663

Train set: Average loss: 0.0078, Accuracy: 73/73 (100%)

tensor(3) tensor(1) tensor(0) tensor(0)
tensor(5) tensor(3) tensor(0) tensor(0)
tensor(9) tensor(3) tensor(0) tensor(0)
tensor(11) tensor(5) tensor(0) tensor(0)
tensor(13) tensor(6) tensor(0) tensor(0)
TP= tensor(13) TN= tensor(6) FN= tensor(0) FP= tensor(0)
TP+FP tensor(13)
precision 1.0
recall 1.0
F1 1.0
acc 1.0
AUC 0.5641025641025641

Test set: Average loss: 0.0198, Accuracy: 19/19 (100%)

Train Epoch: 79 [0/19 (0%)]	Train Loss: 0.000618
Train Epoch: 79 [4/19 (21%)]	Train Loss: 0.006224
Train Epoch: 79 [8/19 (42%)]	Train Loss: 0.019495
Train Epoch: 79 [12/19 (63%)]	Train Loss: 0.007252
Train Epoch: 79 [16/19 (84%)]	Train Loss: 0.012098

Train set: Average loss: 0.0075, Accuracy: 73/73 (100%)

tensor(3) tensor(1) tensor(0) tensor(0)
tensor(5) tensor(3) tensor(0) tensor(0)
tensor(9) tensor(3) tensor(0) tensor(0)
tensor(11) tensor(5) tensor(0) tensor(0)
tensor(13) tensor(6) tens

tensor(11) tensor(4) tensor(0) tensor(1)
tensor(13) tensor(5) tensor(0) tensor(1)
TP= tensor(13) TN= tensor(5) FN= tensor(0) FP= tensor(1)
TP+FP tensor(14)
precision 0.9285714285714286
recall 1.0
F1 0.962962962962963
acc 0.9473684210526315
AUC 0.4615384615384616

Test set: Average loss: 0.0240, Accuracy: 18/19 (95%)

Train Epoch: 90 [0/19 (0%)]	Train Loss: 0.000516
Train Epoch: 90 [4/19 (21%)]	Train Loss: 0.007712
Train Epoch: 90 [8/19 (42%)]	Train Loss: 0.000437
Train Epoch: 90 [12/19 (63%)]	Train Loss: 0.000571
Train Epoch: 90 [16/19 (84%)]	Train Loss: 0.004769

Train set: Average loss: 0.0446, Accuracy: 72/73 (99%)

tensor(3) tensor(0) tensor(0) tensor(1)
tensor(5) tensor(2) tensor(0) tensor(1)
tensor(9) tensor(2) tensor(0) tensor(1)
tensor(11) tensor(4) tensor(0) tensor(1)
tensor(13) tensor(5) tensor(0) tensor(1)
TP= tensor(13) TN= tensor(5) FN= tensor(0) FP= tensor(1)
TP+FP tensor(14)
precision 0.9285714285714286
recall 1.0
F1 0.962962962962963
acc 0.9473684210526315
AUC 0.5256410

tensor(13) tensor(5) tensor(0) tensor(1)
TP= tensor(13) TN= tensor(5) FN= tensor(0) FP= tensor(1)
TP+FP tensor(14)
precision 0.9285714285714286
recall 1.0
F1 0.962962962962963
acc 0.9473684210526315
AUC 0.5256410256410257

Test set: Average loss: 0.0376, Accuracy: 18/19 (95%)

